In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
from sklearn import linear_model

In [2]:
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [3]:
houseprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
id               1460 non-null int64
mssubclass       1460 non-null int64
mszoning         1460 non-null object
lotfrontage      1201 non-null float64
lotarea          1460 non-null int64
street           1460 non-null object
alley            91 non-null object
lotshape         1460 non-null object
landcontour      1460 non-null object
utilities        1460 non-null object
lotconfig        1460 non-null object
landslope        1460 non-null object
neighborhood     1460 non-null object
condition1       1460 non-null object
condition2       1460 non-null object
bldgtype         1460 non-null object
housestyle       1460 non-null object
overallqual      1460 non-null int64
overallcond      1460 non-null int64
yearbuilt        1460 non-null int64
yearremodadd     1460 non-null int64
roofstyle        1460 non-null object
roofmatl         1460 non-null object
exterior1st      1460 non-n

In [4]:
print(houseprices.fireplaces)

0       0
1       1
2       1
3       1
4       1
5       0
6       1
7       2
8       2
9       2
10      0
11      2
12      0
13      1
14      1
15      0
16      1
17      0
18      0
19      0
20      1
21      1
22      1
23      1
24      1
25      1
26      0
27      1
28      2
29      0
       ..
1430    1
1431    0
1432    0
1433    1
1434    1
1435    1
1436    0
1437    1
1438    0
1439    1
1440    1
1441    1
1442    1
1443    1
1444    0
1445    0
1446    0
1447    1
1448    0
1449    0
1450    0
1451    1
1452    0
1453    0
1454    0
1455    1
1456    2
1457    2
1458    0
1459    0
Name: fireplaces, Length: 1460, dtype: int64


In [5]:
houseprices.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', 'firstflrsf', 'secondflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paved

In [6]:
categorical = pd.get_dummies(houseprices, columns = ['street', 'alley', 'utilities', 'centralair','fullbath'], drop_first = True)

In [7]:
import statsmodels.api as sm

X = categorical[['street_Pave','utilities_NoSeWa','centralair_Y','yearbuilt','garagearea','fireplaces']]
Y = categorical['saleprice']
# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     296.8
Date:                Thu, 18 Jul 2019   Prob (F-statistic):          3.21e-248
Time:                        16:23:03   Log-Likelihood:                -17960.
No. Observations:                1460   AIC:                         3.593e+04
Df Residuals:                    1453   BIC:                         3.597e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.458e+06   1.08e+05    -13.493      0.000   -1.67e+06   -1.25e+06
street_Pave       7.107e+04   2.19e+04      3.240      0.001     2.8e+04    1.14e+05
utilities_NoSeWa -5.657e+04   5.34e+04     -1.059      0.290   -1.61e+05    4.82e+04
centralair_Y     -4726.0838   6207.413     -0.761      0.447   -1.69e+04    7450.365
yearbuilt          749.2235     55.610     13.473      0.000     640.138     858.309
garagearea         151.4216      7.667     19.750      0.000     136.383     166.461
fireplaces        3.925e+04   2270.863     17.284      0.000    3.48e+04    4.37e+04
==============================================================================
Omnibus:                      707.707   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8603.342
Skew:                           1.947   Prob(JB):                         0.00
Kurtosis:                      14.237   Cond. No.                     1.57e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?**

Statistically Significant: street_Pave, yearbuilt, garagearea, fireplaces
Not Statistically Significant: utilities_NoSeWa, centralair_Y

In [9]:
A = categorical[['street_Pave','yearbuilt','garagearea','fireplaces']]
Y = categorical['saleprice']
# We need to manually add a constant
# in statsmodels' sm
A = sm.add_constant(A)

results = sm.OLS(Y, A).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     444.8
Date:                Tue, 16 Jul 2019   Prob (F-statistic):          1.70e-250
Time:                        16:05:50   Log-Likelihood:                -17961.
No. Observations:                1460   AIC:                         3.593e+04
Df Residuals:                    1455   BIC:                         3.596e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.437e+06   1.04e+05    -13.864      0.000   -1.64e+06   -1.23e+06
street_Pave  6.983e+04   2.19e+04      3.191      0.001    2.69e+04    1.13e+05
yearbuilt     736.8447     52.766     13.964      0.000     633.339     840.351
garagearea    151.1531      7.662     19.729      0.000     136.124     166.182
fireplaces   3.899e+04   2250.501     17.323      0.000    3.46e+04    4.34e+04
==============================================================================
Omnibus:                      709.288   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8627.831
Skew:                           1.952   Prob(JB):                         0.00
Kurtosis:                      14.251   Cond. No.                     1.51e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Coefficients for the remaining features decrease a little bit. but R-sqaure(total variance covered) stayed similar

**Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?**

If the street is paved, then the house prices would increase by 6969830. If year of the house built increases by 1 unit, then the house prices would increase by 736.84. For every unit of increase in garage area, the house price increase by 151.15. For every unit of increase in fireplaces, the house price would increase by 38990